In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import csv

In [7]:
contents = []

driver = webdriver.Chrome()

wait = WebDriverWait(driver, 10)

# def close_survey_if_present(driver):
#     try:
#         # 최대 10초까지 기다리며 설문조사 팝업의 존재 여부를 확인합니다.
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID, "pop_survey"))
#         )
#         print("설문조사 팝업이 존재합니다. 닫습니다.")
        
#         # 팝업의 닫기 버튼을 클릭합니다.
#         close_button = driver.find_element(By.CSS_SELECTOR, "#pop_survey .btn_close")
#         close_button.click()
        
#         # 팝업이 닫힐 때까지 잠시 대기합니다.
#         WebDriverWait(driver, 10).until(
#             EC.invisibility_of_element_located((By.ID, "pop_survey"))
#         )
#         print("설문조사 팝업이 닫혔습니다.")
        
#     except Exception as e:
#         # 설문조사 팝업이 존재하지 않거나 다른 문제가 발생한 경우
#         print("설문조사 팝업이 없거나 다른 오류가 발생했습니다:", e)

for page in tqdm(range(1, 30)):
    url = f"https://gall.dcinside.com/board/lists/?id=reptile&page={page}"
    driver.get(url)

    # close_survey_if_present(driver)

    try:
        # Wait until the table containing the articles is present
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody")))

        # Determine the range for 'tr' based on the page number
        tr_range = range(4, 51) if page == 1 else range(2, 51)

        for tr in tqdm(tr_range):
            try:
                # Wait for the specific article element to be clickable
                article_locator = (By.CSS_SELECTOR, f"#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({tr}) > td > a")
                article = wait.until(EC.element_to_be_clickable(article_locator))
                article.click()

                # Wait for the content of the article to be present
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > section > article:nth-child(3) > div.view_content_wrap > div > div.inner.clear > div.writing_view_box > div.write_div")))

                soup = bs(driver.page_source, 'html.parser')
                content = soup.select_one("#container > section > article:nth-child(3) > div.view_content_wrap > div > div.inner.clear > div.writing_view_box > div.write_div")
                if content:
                    contents.append(content.text)
                    print(f"Content: {content.text}")

                # Find all comments
                comment_elements = soup.select("div.comment_box > ul > li.ub-content")

                for comment_element in comment_elements:
                    comment_text = comment_element.select_one("div.cmt_txtbox p.usertxt")

                    if comment_text:
                        comment_text = comment_text.text.strip()
                        
                        contents.append(comment_text)
                        print(f"Comment: {comment_text}")

                        # Find all replies to this comment
                        reply_box = comment_element.select_one("div.reply_box")
                        if reply_box:
                            reply_elements = reply_box.select("ul.reply_list > li.ub-content")
                            for reply_element in reply_elements:
                                reply_text = reply_element.select_one("div.cmt_txtbox p.usertxt")
                                
                                if reply_text:
                                    reply_text = reply_text.text.strip()
                                    
                                    contents.append(reply_text)
                                    print(f"Reply: {reply_text}")

                # Go back to the list page and wait for it to load
                driver.back()
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody")))

            except TimeoutException:
                print(f"Timeout while processing article {tr}. Moving to next article.")
            except Exception as e:
                print(f"Error while processing article {tr}: {e}")

    except TimeoutException:
        print(f"Timeout while loading page {page}. Skipping to next page.")
    except Exception as e:
        print(f"Error while loading page {page}: {e}")

# CSV 파일로 저장하기
with open('contents.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Text'])  # 헤더 작성 (필요에 따라 변경 가능)
    
    for item in contents:
        writer.writerow([item])  # 각 아이템을 새 행에 기록

print("CSV 파일이 저장되었습니다.")
driver.quit()


  0%|          | 0/29 [00:00<?, ?it/s]

Content: 
국내에서 사이테스를 그렇게관리했다가 행정력 커버못쳐서 손놓은지경인데 포유류정도면 그렇게 파악가능한데 포유류보다 통상적으로 번식력뛰어난 조류 파충류 어류 절지류는 그게안됨 당장 절지류쪽은 무서류 오너멘탈애들 판치는데 환경부알고도 걍 손놨다 ㅇㅇ 거기 담당공무원 웃긴게 임기가 반년도 아닌데 거의 반년주기로 담당자교체되어있음 티오늘리면 해결이라고 생각할수있겠으나, 애초에 나라가 공무원티오를 전체적으로 줄이는마당에 환경부한테 우선권으로 티오줄리도 없고 사이테스종으로만 한정해도 저지랄인데 전종확대면 절대커버안됨. 그걸 알아서 환경부도 의견좀 받겠다고 공청회를 열었던거고 

Comment: 그렇다고 동물학대를 그냥 냅둘수 있나허가제라도 나와야 무분별한 번식 줄일수있을듯  - dc App
Comment: 크레의정상화를기원합니다다른거도 폭사한다고??함께 폭4하자


Content: 
???: 크레 탈출은 능지순ㅇㅇ???: 제대로 된 크레 가격표 알려줌??????갤 탈출은 능지순 ㅂㅂ~



Content: 
동물학대 멈춰!  - dc official App

Comment: 이상 동물단체도 하는 공청회참석도 안하고 이런데서 자판기밖에 못두드리는 어린학생의 빼액대기 잘봤다
Comment: 크레 정상화 기원합니다


Content: 
그거로 예산편성해서 굴릴수있었으면 진작 외래종들 포상금헌터들이 싹쓸어갔음  포상금이라는 제도 자체가 국가에서 돈주는거라 딴쪽도 쉽게 인정안되는데 그깟 좆충류따위로? ㅋㅋ  



...

 96%|█████████▌| 47/49 [05:04<00:06,  3.01s/it]

Content: 
동생이 도마뱀 키우고 싶다고 용돈 30만원 모아둔걸로 사러가자는데 ㅋㅋ 이거 현실적으로 한두달 겨우 키우다 폐사한다고 봐야함?  - dc official App

Comment: 크레스티드 이거 사자는데 초보자가 1년 이상 키우는거 불가능이라고 봐야대지?  - dc App
Comment: 아무렴 너도 키웟는데 도마뱀은 못키워  - dc App


Content: 
이새끼 ㄹㅇ뭐냐.. 개쫄았다진짜  - dc official App

Comment: 구렁이 아나콘다 실제로 본적은 있음?
Comment: 아나콘다 ㅇㅈㄹㅋㅋㅋㅋ  - dc App


100%|██████████| 29/29 [1:09:50<00:00, 144.51s/it]


CSV 파일이 저장되었습니다.


In [8]:
print(len(contents))

5110
